In [2]:
from ipsl_dcpp.model.ipsl_dataset import IPSL_DCPP
import torch
import lightning as pl
from ipsl_dcpp.model.pangu import PanguWeather
from hydra import compose, initialize
from omegaconf import OmegaConf

import hydra
import os
os.environ['SLURM_NTASKS_PER_NODE'] = '1'

work_dir = os.environ['WORK']
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")
pl.seed_everything(cfg.experiment.seed)
train = hydra.utils.instantiate(
    cfg.experiment.train_dataset,
    generate_statistics=False,
    surface_variables=cfg.experiment.surface_variables,
    depth_variables=cfg.experiment.depth_variables,
    plev_variables=cfg.experiment.plev_variables,
    normalization='climatology',
    delta=True
)

train_dataloader = torch.utils.data.DataLoader(
    train,
    batch_size=1,
    shuffle=True,
    num_workers=1
)

#batch = next(iter(train_dataloader))
model = hydra.utils.instantiate(
    cfg.experiment.module,
    backbone=hydra.utils.instantiate(
        cfg.experiment.backbone,
    ),
    dataset=train_dataloader.dataset
)
trainer = pl.Trainer(
    max_epochs=cfg.experiment.max_epochs,
    enable_checkpointing=True,
    log_every_n_steps=1,
   # max_steps=cfg.experiment.max_steps if not cfg.debug else 10,
    #precision="16-mixed",
    #precision='32',
    profiler='simple' if cfg.debug else None,
   # devices=cfg.experiment.num_gpus,
   # strategy='ddp_find_unused_parameters_true',
    #limit_train_batches=0.01 if cfg.debug else 1
    #limit_val_batches=0.01 if cfg.debug else 1,
    fast_dev_run=1
)

model = hydra.utils.instantiate(
    cfg.experiment.module,
    backbone=hydra.utils.instantiate(
        cfg.experiment.backbone,
    ),
    dataset=train_dataloader.dataset
)

[rank: 0] Seed set to 0
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/torch/nn/init.py:412: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a 

In [ ]:
trainer.fit(
    model=model,
    train_dataloaders=train_dataloader)
trainer.logged_metrics

In [54]:
from ipsl_dcpp.model.embedding import PatchEmbed2D,PatchEmbed3D
import torch.nn as nn
surface1 = batch['state_surface'].squeeze(-4)
# print('surface',surface)
# print('surface non nan',torch.nonzero(torch.isnan(surface.view(-1))))
# upper_air = batch['state_level']
depth = batch['state_depth'].squeeze(-5)
patchembed2d = PatchEmbed2D(
            img_size=(143, 144),
            patch_size=(2,2),
            in_chans=91,  
            embed_dim=192,
        )
depth_patchembed3d = PatchEmbed3D(
            img_size=(11, 143, 144),
            patch_size=(2,2,2),
            in_chans=3,
            embed_dim=192
        )
surface = patchembed2d(surface1)
depth = depth_patchembed3d(depth)

x1 = torch.concat([surface.unsqueeze(2),depth], dim=2)
B, C, Pl, Lat, Lon = x1.shape

x = x1.reshape(B, C, -1).transpose(1, 2)
norm1 = nn.LayerNorm(192,eps=1e-10)
normed = norm1(x)